In [1]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, preprocessing, linear_model, model_selection


import lightgbm as lgb
from sklearn.linear_model import Lasso
from xgboost import XGBClassifier


from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, SpectralClustering, MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

from sklearn.model_selection import train_test_split


import seaborn as sm
import matplotlib.pyplot as plotGraph
from sklearn.decomposition import PCA, FactorAnalysis


from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

from yellowbrick.cluster import KElbowVisualizer

from sklearn.preprocessing import RobustScaler, MaxAbsScaler, PowerTransformer, Normalizer, StandardScaler, QuantileTransformer
from scipy.sparse import csr_matrix
from tqdm import trange



In [2]:
data_df = pd.read_csv("data.csv")
submission_csv = pd.read_csv("sample_submission.csv")

In [3]:
data_df

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
0,0,-0.389420,-0.912791,0.648951,0.589045,-0.830817,0.733624,2.258560,2,13,...,-0.478412,-0.757002,-0.763635,-1.090369,1.142641,-0.884274,1.137896,1.309073,1.463002,0.813527
1,1,-0.689249,-0.453954,0.654175,0.995248,-1.653020,0.863810,-0.090651,2,3,...,-0.428791,-0.089908,-1.784204,-0.839474,0.459685,1.759412,-0.275422,-0.852168,0.562457,-2.680541
2,2,0.809079,0.324568,-1.170602,-0.624491,0.105448,0.783948,1.988301,5,11,...,-0.413534,-1.602377,1.190984,3.267116,-0.088322,-2.168635,-0.974989,1.335763,-1.110655,-3.630723
3,3,-0.500923,0.229049,0.264109,0.231520,0.415012,-1.221269,0.138850,6,2,...,0.619283,1.287801,0.532837,1.036631,-2.041828,1.440490,-1.900191,-0.630771,-0.050641,0.238333
4,4,-0.671268,-1.039533,-0.270155,-1.830264,-0.290108,-1.852809,0.781898,8,7,...,-1.628830,-0.434948,0.322505,0.284326,-2.438365,1.473930,-1.044684,1.602686,-0.405263,-1.987263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,97995,0.237591,1.657034,-0.689282,0.313710,-0.299039,0.329139,1.607378,5,7,...,-0.290116,-0.258141,-0.973640,1.369508,0.391055,2.152426,-0.208944,-1.475403,0.298448,0.445039
97996,97996,0.322696,0.710411,0.562625,-1.321713,-0.357708,0.182024,0.178558,3,9,...,0.117687,1.388242,0.342400,1.680537,-0.860409,0.579165,1.162692,0.134994,0.994666,0.727642
97997,97997,-0.249364,-0.459545,1.886122,-1.340310,0.195029,-0.559520,-0.379767,8,9,...,-0.850223,-1.787648,-1.268115,-1.508330,1.945622,1.503645,0.194968,2.142693,1.646042,0.641466
97998,97998,0.311408,2.185237,0.761367,0.436723,0.464967,0.062321,-0.334025,1,8,...,-0.010839,1.118906,1.565765,0.358480,0.547615,1.224439,-0.537998,-1.610954,-0.616227,-0.066211


In [4]:
# # find optimal number of neighbors using cross validation
# figure, axis = plotGraph.subplots(figsize=(20, 10))



# scores_array=[]
# i_track = []
# for i in range(1, 21):
#     kmeans_ = KMeans(n_clusters = i, random_state=0)
#     cvs = cross_val_score(kmeans_, data_df, cv=10)
#     avg_cvs_score = np.mean(cvs)
#     print(kmeans_,':', np.abs(avg_cvs_score))
#     i_track.append(i)
#     scores_array.append(  np.abs(avg_cvs_score))
#     axis.scatter(i,np.abs(avg_cvs_score))
#     axis.annotate('(N=' +str(i) +', '+ str(round(avg_cvs_score,3)) +')', (i, avg_cvs_score), textcoords='offset points', xytext=(0,20), ha='center')



# axis.plot(i_track, scores_array)

# axis.set_title('Title: Finding Optimal Neighbors', fontsize=15)
# axis.set_ylabel('Average Cross Validation Score', fontsize=15)
# axis.set_xlabel('Neighbors', fontsize=15)

In [5]:
col = list(data_df.columns)
col

['id',
 'f_00',
 'f_01',
 'f_02',
 'f_03',
 'f_04',
 'f_05',
 'f_06',
 'f_07',
 'f_08',
 'f_09',
 'f_10',
 'f_11',
 'f_12',
 'f_13',
 'f_14',
 'f_15',
 'f_16',
 'f_17',
 'f_18',
 'f_19',
 'f_20',
 'f_21',
 'f_22',
 'f_23',
 'f_24',
 'f_25',
 'f_26',
 'f_27',
 'f_28']

In [6]:
# best_data =[
# 'f_07','f_08', 'f_09', 'f_10',
# 'f_11', 'f_12', 'f_13', 'f_22',
# 'f_23', 'f_24', 'f_25','f_26',
# 'f_27', 'f_28']

In [7]:
# rs = RobustScaler()

In [8]:
# nm = Normalizer().fit_transform(data_df)

In [9]:
# s_scaled = StandardScaler().fit_transform(nm)

In [10]:
# rs_scaled = rs.fit_transform(s_scaled)

In [11]:
# abs_scaled = MaxAbsScaler().fit(data_df)

In [12]:
data_standard = MaxAbsScaler().fit_transform(data_df)

In [13]:
power_transformer = PowerTransformer().fit(data_standard)

In [14]:
data_transformed = power_transformer.transform(data_standard)

In [15]:
# weakened score
# data_transformed = QuantileTransformer(output_distribution = "normal").fit_transform(data_standard)

In [16]:
# weakened score
# data_transformed = FactorAnalysis().fit_transform(data_transformed)

In [17]:
data_transformed = pd.DataFrame(data_transformed, columns = col)

In [18]:
data_transformed

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
0,-1.785610,-0.389214,-0.917595,0.648026,0.590794,-0.824839,0.734726,2.258634,-1.025849,1.416210,...,-0.472796,-0.753837,-0.763138,-0.707687,0.911286,-0.678769,0.768872,0.960589,1.044229,0.693363
1,-1.785571,-0.688343,-0.458551,0.653256,0.995491,-1.644095,0.864904,-0.085610,-1.025849,-0.917129,...,-0.423449,-0.088090,-1.778004,-0.535924,0.453141,1.031995,-0.120019,-0.551558,0.367193,-1.632988
2,-1.785532,0.805736,0.319496,-1.167199,-0.622338,0.108418,0.785046,1.990036,0.000862,1.044870,...,-0.408275,-1.598766,1.194129,2.204238,0.086137,-1.518918,-0.568980,0.979203,-0.925330,-2.295500
3,-1.785493,-0.500449,0.224093,0.262979,0.234051,0.417077,-1.218870,0.144455,0.285546,-1.235141,...,0.620343,1.283814,0.533013,0.731742,-1.217743,0.826924,-1.171309,-0.396198,-0.101020,0.323597
4,-1.785454,-0.670402,-1.044451,-0.270555,-1.833519,-0.285916,-1.849552,0.787766,0.786547,0.175188,...,-1.615251,-0.432316,0.322156,0.227003,-1.481657,0.848441,-0.614021,1.165287,-0.374743,-1.156993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97995,1.686011,0.236090,1.648820,-0.688289,0.316108,-0.294819,0.330327,1.611017,0.000862,0.175188,...,-0.285507,-0.255894,-0.972327,0.953666,0.407148,1.284268,-0.077663,-0.989422,0.166338,0.457077
97996,1.686044,0.320937,0.704710,0.561603,-1.322057,-0.353300,0.183266,0.184235,-0.651268,0.633441,...,0.120435,1.383688,0.342091,1.160290,-0.429981,0.271377,0.784262,0.140360,0.693639,0.638477
97997,1.686077,-0.249510,-0.464142,1.889491,-1.340744,0.197732,-0.557804,-0.375800,0.786547,0.633441,...,-0.842363,-1.784109,-1.265335,-0.994906,1.450956,0.867559,0.178499,1.541355,1.180404,0.583292
97998,1.686110,0.309684,2.175086,0.760609,0.438870,0.466890,0.063617,-0.329863,-1.438508,0.410645,...,-0.007552,1.115833,1.571433,0.276960,0.512080,0.687810,-0.287849,-1.084754,-0.538573,0.125657


In [19]:
# helped increase kaggle score by 40%
# rs = RobustScaler()

In [20]:
# rs_scaled = rs.fit_transform(data_df)

In [21]:
# pca_data = PCA(n_components = .95, random_state=0)

In [22]:
# weakened score
# pca_data_tranformed = pca_data.fit_transform(data_transformed)

In [23]:
# ElbowV = KElbowVisualizer(KMeans(random_state=0), k=(4,21))
# ElbowV.fit(pca_data_tranformed)
# ElbowV.show()

In [24]:
# ElbowV = KElbowVisualizer(KMeans(random_state=0), k=(4,21))
# ElbowV.fit(data_transformed)
# ElbowV.show()

In [25]:
# clusters_final = BayesianGaussianMixture(n_components=10, covariance_type="full", random_state=0).fit_predict(pca_data_tranformed)

In [26]:
# terrible - score of .1
# clusters_final = MiniBatchKMeans(n_clusters=10, batch_size=64, n_init=1, random_state=0).fit_predict(pca_data_tranformed)

In [27]:
# causing kernal dying
# clusters_final = AgglomerativeClustering(n_clusters=10).fit_predict(pca_data_tranformed)

In [28]:
# causing kernal dying
# clusters_final = SpectralClustering(n_clusters=10).fit_predict(pca_data_tranformed)

In [29]:
# score is 0
# clusters_final = MeanShift().fit_predict(pca_data_tranformed)

In [30]:
# score 44
# clusters_final = GaussianMixture(n_components=10, covariance_type="full", random_state=0).fit_predict(pca_data_tranformed)

In [31]:
# did not improve score
# clusters_final = BayesianGaussianMixture(n_components=10, covariance_type="full", random_state=42).fit_predict(pca_data_tranformed)

In [32]:
# n_init=3 did not change score of 0.59796
clusters_final = BayesianGaussianMixture(n_components=7, random_state=0).fit_predict(data_tranformed)

/Users/myplayhouse/opt/anaconda3/lib/python3.8/site-packages/sklearn/mixture/_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '


In [33]:
clusters_final.shape

(98000,)

In [34]:
submission_csv["Predicted"] = clusters_final

In [35]:
submission_csv

,Id,Predicted
0,0,2
1,1,1
2,2,6
3,3,4
4,4,3
...,...,...
97995,97995,1
97996,97996,3
97997,97997,6
97998,97998,0


In [36]:
submission_csv.to_csv("july_submission_20.csv", index=False)